In [ ]:
pip install tensorflow numpy matplotlib

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, callbacks

# Enable Mixed Precision for Faster Computation (If GPU Available)
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# ✅ Load & Normalize CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to range [0,1]

# ✅ Convert Data into Efficient TensorFlow Pipelines
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(64).prefetch(tf.data.AUTOTUNE)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64).prefetch(tf.data.AUTOTUNE)

# ✅ Step 1: Define CNN Model Function (Ensures Model is Created)
def build_cnn_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(10, activation='softmax')  # Output layer for 10 classes
    ])
    return model
# ✅ Step 2: Initialize & Compile the Model (Now 'cnn_model' Exists)
cnn_model = build_cnn_model()

cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

In [ ]:
# ✅ Step 3: Implement Early Stopping (Prevents Overfitting)
early_stopper = callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
# ✅ Step 4: Train Model (No NameError Now!)
training_progress = cnn_model.fit(train_data,
                                  epochs=5,  # Reduced Epochs for Faster Training
                                  validation_data=test_data,
                                  callbacks=[early_stopper])

# ✅ Step 5: Evaluate Model Performance
test_loss, test_accuracy = cnn_model.evaluate(test_data, verbose=2)
print(f"Final Model Accuracy: {test_accuracy * 100:.2f}%")

# ✅ Step 6: Plot Training vs Validation Accuracy & Loss
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(training_progress.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(training_progress.history['val_accuracy'], label='Validation Accuracy', marker='s')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(training_progress.history['loss'], label='Train Loss', marker='o')
plt.plot(training_progress.history['val_loss'], label='Validation Loss', marker='s')
plt.legend()
plt.title('Model Loss')

plt.show()
